In [ ]:
%load_ext autoreload
%autoreload 2

# Sonecule: ContinuousPMS – Continuous Parameter-Mapping Sonification

This notebook introduces and demonstrates usage of the ContinousPMS sonecule.
* The sonecule uses a synth to create a continuous sound stream 
* for that a mutable synth is used that offers several control parameters for modulation
* Specifically the (default) parameters are:
  * amplitude
  * frequency
  * sharpness of the sound
  * spatial panning
* Most likely, a custom synth will be created and passed on for individual sonifications, replacing the default.
* The mapping specifies how data channels shall control the individual parameters

In [ ]:
# headers and imports for the demo
import sonecules as sn
from pya import Asig
import pyamapping as pam
import matplotlib.pyplot as plt

# setup for matplotlib 
plt.rcParams["figure.figsize"] = (8,3)
%matplotlib widget

# start sonecules (with default backend sc3nb, aka sc3)
sn.startup()
ctx = sn.gcc()  # get the context as ctx

Load data sets used for the demo

In [ ]:
%run ../data/prepare-data.ipynb

For this sonecule we work with the building data set for the demonstrations. That is a data set that describes the hourly water and electricity consumption of an office building plus environmental features such as temperature, humidity, wind-speed, time of day, solar radiation. With roughly 4300 rows the data set contains roughly half a year of data.

In [ ]:
df = dataframes['building']
df.iloc[:,10:-1].plot(lw=0.5, subplots=True, figsize=(9,6)); plt.tight_layout()
df.head()

## Usage Demo for the ContinuousPMS Sonecule

In [ ]:
from sonecules.scoreson import ContinuousPMS, pms

The following code cell shows everything needed 
- to create the sonecule with data, 
- to reset the auditory canvas (aka timeline)
- to start the playback at a given rate
- to plot the timeline.

Usually we want to use a specific synths which we would define using the backend.
- We can create it using `context.synths.add_synth_def`, which in this case creates the defines the synth for the default backend (sc3nb)
- The synth definition process is currently in active development and will improve in the future.  
- However our ultimate goal is that sonecules will already offer a well curated library of pre-packaged synths, so that it is rarely necessary to craft your own.


The default synth (if none is provided is "cpmssyn", a 'continuous synth for PMSon'). 
- It is a pitched tone with added vibrato 
- It offers the continuous controls:

| parameter   | range  | meaning                                         |
| ----------- | ------ | ----------------------------------------------- |
| freq:       | [20..] | frequency                                       |
| amp         | [0,1]  | amplitude                                       |
| sharp: | [0,10] | sharp, the higher the more sharp the sound |
| pan         | [-1,1] | spatial panning from left to right              |
| plfreq      | [0..]  | multiplied pulse frequency                      |
| plwid       | [0,1]  | duty cycle of the pulse                         |
| plint       | [0,1]  | intensity of the pulse modulation               |

As currently no default synths are implemented we have to do the work ourselves

In [ ]:
# a pulsed synth with sharpness control
ctx.synths.add_synth_def("syc0", 
r"""{ | out=0, freq=400, amp=0.1, plfreq=0, plwid=0.5, plint=0, sharp=0, pan=0, lg=0 | 
    var f = freq.lag(lg);
    var pulse = LFPulse.ar(plfreq, width: plwid, mul: plint, add: 1-plint);
    var tone = HPF.ar(Formant.ar(f, f, bwfreq: f * (sharp + 1)), 40);
    Out.ar(out, Pan2.ar(tone*pulse, pan.lag(lg), level: amp.lag(lg)));
}""")

# a continuous synth with sharpness and vibrate controls
ctx.synths.add_synth_def("sycvib", 
r"""{ | out=0, freq=400, amp=0.1, vibfreq=0, vibintrel=0, sharp=0, pan=0, lg=0 | 
    var vib = SinOsc.ar(vibfreq, mul: vibintrel*freq, add: freq.lag(lg));
    var sig = HPF.ar(Formant.ar(vib, vib, bwfreq: vib*sharp+1, mul: amp.lag(lg)), 40);
    Out.ar(out, Pan2.ar(sig, pan.lag(lg)));
}""")

# a filtered noise synths with center frequency freq and reciprocal quality (rq) control 
ctx.synths.add_synth_def("sycnoise", 
r"""{ | out=0, freq=400, amp=0.1, rq=0.1, pan=0, lg=0 | 
    var sig = BPF.ar(WhiteNoise.ar(amp), freq.lag(lg), rq.lag(lg));
    Out.ar(out, Pan2.ar(sig, pan.lag(lg)));
}""")

# filtered impulse train with filter and freq controls
ctx.synths.add_synth_def("syctick", 
r"""{ | out=0, freq=400, cf=4000, amp=0.1, pan=0, lg=0 | 
    var sig = LPF.ar(Impulse.ar(freq.lag(lg)), cf.lag(lg));
    Out.ar(out, Pan2.ar(sig, pan.lag(lg), amp));
}""")


In [ ]:
from ipywidgets import interactive

In [ ]:
ctx.enable_realtime() # we enable realtime for some Synth demos

Here some mesonic interactions to explore and play with the above defined synths

In [ ]:
ctx.reset(0)
# use this code to explore the synths as you see fit
sx = ctx.synths.create("syc0")
sx.start(freq=200, amp=0.1, plfreq=5, plwid=0.1, sharp=4, pan=0)

In [ ]:
sx.freq = 300
sx.amp = 0.1
sx.plfreq = 15
sx.plwid = 0.7
sx.plint = 0.5
sx.sharp = 2
sx.pan = 0
sx.lg = 0

In [ ]:
sx.stop()

In [ ]:
ctx.reset()
syctick = ctx.synths.create("syctick")
syctick.start(freq=80, amp=0.05, cf=2000, pan=0)
def syn_gui(freq=10, cf=4000, amp=0.15, pan=0, lg=0.5):
    syctick.set(freq=freq, cf=cf, amp=amp, pan=pan, lg=lg)
interactive(syn_gui, freq=(0, 1000, 1), cf=(1, 20000, 1), amp=(0, 1, 0.01), pan=(-1,1,0.1), lg=(0,5,0.1)) 

In [ ]:
syctick.stop()

In [ ]:
ctx.reset()
# demo for the sycnoise
sx = ctx.synths.create("sycnoise")
with ctx.at(0): sx.start(freq=800, amp=0.05, rq=0.1, pan=0)
with ctx.at(1): sx.set(lg=0.5, freq=500,  rq=0.1, amp=0.2, pan=-1)
with ctx.at(2): sx.set(lg=0.5, freq=5000, rq=0.5, amp=0.1, pan=1)
with ctx.at(3): sx.set(lg=0.5, freq=2000, rq=1.2, amp=0.1, pan=1) 
with ctx.at(4): sx.set(lg=0.5, freq=400,  rq=0.1, amp=0.3, pan=1) 
with ctx.at(5): sx.stop()

In [ ]:
ctx.reset()
ctx.disable_realtime()

Let's start with Parameter Mapping Sonification

The next cell shows all steps in specifying the mapping in one go.

- We use 10 days, i.e. 10*24 hours of the dataset
- we map only those values needed using test_mapping
- we set default values (as needed) in the sonecule constructor

You will hear the ups and downs of the humidity over 10 days in 4 seconds

In [ ]:
# create a sonecule, initialized with the synth to be used
# second argument can provide defaults and bounds for parameters
sncpms = ContinuousPMS("syc0", { 
    "freq" : {"bounds": [100, 5000]},
    "amp"  : {"default": 0.1},
    "sharp": {"default": 0},
    "lg"   : {"default": 0.1}
})

# the mapping is just a dictionary where keys are the parameters,
# and values are dictionaries with keys tuples of 
# - data column, 
# - mapping type, and  
# - arguments of the mapping function as dictionary, e.g.
test_mapping = {
    "onset": ("INDEX", "lin", [0, 4]),
    "freq" : ("humidity", "exp", [pam.midi_to_cps(50), pam.midi_to_cps(70)]),
    "sharp": ("temperature", "lin", [0, 10]),
    "pan"  : ("solar_radiation", "lin", [-1, 1]),
}

# reset the timeline 
ctx.timeline.reset() 

# and render the sonification into the timeline
sncpms.schedule(df=df.iloc[:10*24], mapping=test_mapping)

# finally start the realtime playback at a given rate
sncpms.start(rate=1)

The mapping is just a dictionary where 
- keys are the parameters, and 
- values is either **a number** (for a constant) or **a dictionary** with some mandatory and some optional keys:
    - `"col"`: column/feature of the dataset to be used 
    - `"fn"`: mapping function(values, xr, yr), but strings such as `"lin"`, `"exp"`, `"log"`
        are allowed as shortcut
    - `"yr"`: the target range for the parameter (implied from synths bounds if omitted)
    - `"xr"`: the source range (which is implied from the data if omitted)
see documenation for further mapping flags. See the [DiscretePMS](sonecule-DiscretePMS.ipynb) for more details on the mapping specification

Usually this can be done in a more condensed way, by
- omitting the defaults and bounds (later synths will come with good such values anyway)
- using shortcuts such as providing `[min, max]` instead of `{"y1": min, "y2": max}`
- setting constant values by value
- defining the mapping in the call
- starting the sonification by daisy chaining
as shown here

In [ ]:
# create a sonecule, initialized with your data selection
sncpms = ContinuousPMS("syc0")
ctx.timeline.reset() 
df_selection = df.iloc[:14*24]
sncpms.schedule(df=df_selection, mapping=dict(
    onset = pms("INDEX",           "lin", [0, 10]),
    freq  = pms("humidity",        "exp", [100, 400]),
    sharp = pms("temperature",     "lin", [2, 8]),
    pan   = pms("solar_radiation", "lin", [-1, 1]),
    amp   = pms("wind_speed",      "exp", [0.02, 1]),
    lg    = 0.05,
))

# lets plot the used data
df_selection.loc[:, ['humidity', 'temperature', 'solar_radiation', 'wind_speed']].plot()

sncpms.start()

Note that the events remain in the timeline. 

In [ ]:
ctx.timeline

In [ ]:
# if needed check entries using 
# ctx.timeline.to_dict()

Starting the playback will result in the Timeline to be replayed

In [ ]:
ctx.playback.start(at=0)  # or ctx.playback.time = 0 if ctx.playback.running 

Note that starting at a time where the Synth was not playing results in errors at the backend.

In [ ]:
ctx.playback.start(at=9.5)  # or ctx.playback.time = 0 if ctx.playback.running 

For reseting timeline back to the empty state

In [ ]:
ctx.timeline.reset()

In [ ]:
ctx.timeline

In [ ]:
# to stop all sound playing via the backend use 
ctx.stop()

In [ ]:
# for own mapping experiments, its useful to see all columns
df.columns

In [ ]:
# as long as we reuse the synth, no need to create the object, but just use with different mappings
sncpms.reschedule(df=df.iloc[4*24:14*24], mapping=dict(
    onset = pms("INDEX",           "lin", [0, 4]),
    freq  = pms("humidity",        "exp", [100, 400]),
    sharp = pms("temperature",     "lin", [2, 8]),
    pan   = pms("solar_radiation", "lin", [-1, 1]),
    amp   = pms("wind_speed",      "exp", [0.02, 1]),
    lg    = 0.05,
)).start(rate=1)

### Hyrbid Continuous Parameter Mappings

Let's now explore a more interesting mapping:
- together with the above mapping of s
- to play impulses for the electricity
- wind sounds for the wind speed
- and map the water use on pulse choppings of the stream

In [ ]:
# create a sonecule, initialized with your data selection
df_selection = df.iloc[0*24:14*24]
df_selection.iloc[:,10:-2].plot(subplots=True, figsize=(10,6));

In [ ]:
# test does ctx.reset before the indented code and ctx.playback.start after 
with ctx.test():
    onset_mapping_spec = pms("INDEX", "lin", [0, 10])
    s1 = ContinuousPMS("syc0").schedule(df_selection, dict(
        onset = onset_mapping_spec,
        freq  = pms("temperature", "lin", [50, 62], post="midicps"),
        sharp = pms("humidity", "lin", [2, 8]),
        amp   = pms("solar_radiation", "lin", [-30, 0], post="dbamp"),
        pan = 0, lg = 0.05,
    ))
    s2 = ContinuousPMS("sycnoise").schedule(df_selection, dict(
        onset = onset_mapping_spec,
        freq  = pms("wind_speed", "exp", [100, 2000]),
        amp   = pms("wind_speed", "exp", [0.2, 0.8]),
        pan = 0.5, rq = 0.4, lg = 0.05,
    ))
    s3 = ContinuousPMS("syctick").schedule(df_selection, dict(
        onset = onset_mapping_spec,
        freq  = pms("hc_wb_electrical", "exp", [10, 80]),
        pan = -0.5, cf = 8000, amp = 1, lg = 0.05,
    ))

In [ ]:
# Remember that you can start the playback again to listen again
ctx.playback.start()

In [ ]:
ctx.timeline.plot()

## Code Template

The following code snippets are intended for copy & paste to your notebooks, to facilitate getting your data sonified
using this sonecule.

In [ ]:
# load your data / select your data
data = np.random.random((1000, 4)) - 0.5 # 100 rows with 8 channels, here same fake data
data = np.cumsum(data,axis=0)
df = pd.DataFrame(data, columns=["c1", "c2", "c3", "c4"])
df.plot(subplots=True);

In [ ]:
ctx.reset()

In [ ]:
# prepare your Synth Definition 
# This will be improved in the future. 
ctx.synths.add_synth_def("syc0", 
"""{ | out=0, freq=400, amp=0.1, plfreq=0, plwid=0.5, plint=0, sharp=0, pan=0, lg=0 | 
    var f = freq.lag(lg);
    var pulse = LFPulse.ar(plfreq, width: plwid, mul: plint, add: 1-plint);
    var tone = HPF.ar(Formant.ar(f, f, bwfreq: f * (sharp + 1)), 40);
    Out.ar(out, Pan2.ar(tone*pulse, pan.lag(lg), level: amp.lag(lg)));
}""")

# sonecule for your synth with defaults and bounds
sncpms = ContinuousPMS("syc0", { 
    "freq": {"bounds": (100, 5000)},
    "amp": {"default": 0.5},
    "sharp": {"default": 0},
    "lg": {"default": 0.01}
    })

In [ ]:
# mapping dictionary 
test_mapping = dict(
    onset  = pms("INDEX", "lin", [0, 8]),
    freq   = pms("c1", "exp", [pam.midi_to_cps(50), pam.midi_to_cps(70)]),
    sharp  = pms("c2", "lin", [0, 10]),
    plfreq = pms("c3", "lin", [5, 25]),
    plint  = 1, plwid = 0.5,
)

# and render the sonification into the timeline
sncpms.reschedule(df=df, mapping=test_mapping)

# finally start the realtime playback at a given rate
sncpms.start(rate=1)

In [ ]:
# if needed: plot the timeline using 
ctx.timeline.plot()

In [ ]:
ctx.close()